In [ ]:
%matplotlib inline

## Iris introduction course
# 3. Subcube Extraction

**Learning outcome**: by the end of this section, you will be able to apply Iris functionality to take a useful subset of an Iris cube.

**Duration:** 1 hour

**Overview:**<br>
3.1 [Indexing](#indexing)<br>
3.2 [Constraints and Extraction](#constrain_extract)<br>
3.3 [Iterating Over a Cube](#iteration)<br>
3.4 [Summary of the Section](#summary)

## Setup

In [ ]:
import iris

----

## 3.1 Indexing<a id='indexing'></a>

Cubes can be indexed in a familiar manner to that of NumPy arrays:

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname, 'air_potential_temperature')
print(cube.summary(shorten=True))

In [ ]:
subcube = cube[..., ::2, 15:35, :10]
subcube.summary(shorten=True)

Note: the result of indexing a cube is *always* a copy and never a *view* on the original data.

----

## 3.2 Constraints and Extraction<a id='constrain_extract'></a>

We've already seen the basic ``load`` function, but we can also control which cubes are actually loaded with *constraints*. The simplest constraint is just a string, which filters cubes based on their name:

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
print(iris.load(fname, 'air_potential_temperature'))

Iris's constraints mechanism provides a powerful way to filter a subset of data from a larger collection. We've already seen that constraints can be used at load time to return data of interest from a file, but we can also apply constraints to a single cube, or a list of cubes, using their respective ``extract`` methods.

In [ ]:
cubes = iris.load(fname)
print(cubes.extract('air_potential_temperature'))

The simplest constraint, namely a string that matches a cube's name, is conveniently converted into an actual ``iris.Constraint`` instance wherever needed. However, we could construct this constraint manually and compare with the previous result:

In [ ]:
pot_temperature_constraint = iris.Constraint('air_potential_temperature')
print(cubes.extract(pot_temperature_constraint))

The Constraint constructor also takes arbitrary keywords to constrain coordinate values. For example, to extract model level number 10 from the air potential temperature cube:

In [ ]:
pot_temperature_constraint = iris.Constraint('air_potential_temperature',
                                             model_level_number=10)
print(cubes.extract(pot_temperature_constraint))

We can pass a list of possible values, and even combine two constraints with ``&``:

In [ ]:
print(cubes.extract('air_potential_temperature' & 
                    iris.Constraint(model_level_number=[4, 10])))

We can define arbitrary functions that operate on each cell of a coordinate. This is a common thing to do for floating point coordinates, where exact equality is non-trivial.

In [ ]:
def less_than_10(cell):
    """Return True for values that are less than 10."""
    return cell < 10

print(cubes.extract(iris.Constraint('air_potential_temperature',
                                    model_level_number=less_than_10)))

### Time Constraints<a id='time_constraints'></a>

It is common to want to build a constraint for time.  
This can be achieved by comparing cells containing datetimes

There are a few different approaches for producing time constraints in Iris. We will focus here on one approach for constraining on time in Iris. 

This approach allows us to access individual components of cell datetime objects and run comparisons on those:

In [ ]:
time_constraint = iris.Constraint(time=lambda cell: cell.point.hour == 11)
print(cube.extract(time_constraint).summary(True))

### Exercise 2

Cell methods are a part of cube metadata that record statistical operations that have been applied to a cube. For example, "`mean: time (6hrs)`" tells us that the cube has had a time mean over a 6hr interval applied.

We can determine what, if any, cell methods a cube has with the attribute `cube.cell_methods`. The following function, then, tells us whether or not a cube has cell methods:

```python
def has_cell_methods(cube):
    return len(cube.cell_methods) > 0
```

1\. With the cubes loaded from ``[iris.sample_data_path('A1B_north_america.nc'), iris.sample_data_path('uk_hires.pp')]`` use the CubeList's **``extract``** method to filter only the cubes that have cell methods. (Hint: Look at the ``iris.Constraint`` documentation for the **cube_func** keyword). You should find that the 3 cubes are whittled down to just 1.

2\. Using the file found at ``iris.sample_data_path('A1B_north_america.nc')`` filter the cube, using constraints, such that only data between 1860 and 1980 remains (hint: This data has a 360-day calendar with yearly data from 1860 to 2100, so we will need to access the individual components of the cell point's datetime, to return a time dimension of length 120).

----

## 3.3 Iterating Over a Cube<a id='iteration'></a>

We can loop through all desired subcubes in a larger cube using the cube methods ``slices`` and ``slices_over``.

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname,
                      iris.Constraint('air_potential_temperature',
                                      model_level_number=1))
print(cube.summary(True))

The **``slices``** method returns all the slices of a cube on the dimensions specified by the coordinates passed to the slices method.

So in this example, each `grid_latitude` / `grid_longitude` slice of the cube is returned:

In [ ]:
for subcube in cube.slices(['grid_latitude', 'grid_longitude']):
    print(subcube.summary(shorten=True))

We can use **``slices_over``** to return one subcube for each coordinate value in a specified coordinate. This helps us when trying to retrieve all the slices along a given cube dimension.

For example, let's consider retrieving all the slices over the time dimension (i.e. each time step in its own cube with a scalar time coordinate) using ``slices``. As per the above example, to achieve this using ``slices`` we would have to specify all the cube's dimensions _except_ the time dimension.

Let's take a look at ``slices_over`` providing this functionality:

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname, 'air_potential_temperature')
for subcube in cube.slices_over('model_level_number'):
    print(subcube.summary(shorten=True))

----

## 3.4 Section Summary : Subcube Extraction<a id='summary'></a>

In this section we learnt:
* cubes can be indexed like numpy arrays to produce sub-cubes
* 'constraint' objects can be used to load only part of the data
* particular methods are used to extract data by dates and times
* a cube can be "sliced up" along some of its dimensions, looping over all the possible subcube 'slices'.
